In [4]:
import os
import sys
import pdb
import glob
import numpy as np
import scipy.interpolate
import scipy.io
import matlab.engine
import importlib
import soundfile as sf
import multiprocessing
import functools

import matplotlib.pyplot as plt

import simulator
importlib.reload(simulator)

import simulator_matlab
importlib.reload(simulator_matlab)

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_figures
import util_stimuli
import util_misc


In [70]:
def load_kemar_hrtfs(npz_filename='kemar_hrtfs/hrtfs.npz'):
    """
    Helper function to load KEMAR HRTFs from Gardner & Martin (1994).
    Source: https://sound.media.mit.edu/resources/KEMAR.html
    
    Returns
    -------
    hrtf_locs (float array w/ shape [368, 3]): polar (1.4m, azim, elev)
    hrtf_firs (float array w/ shape [368, 128, 2]): impulse responses
    hrtf_sr (int): sampling rate of impulse responses (44100 Hz)
    """
    if os.path.exists(npz_filename):
        f = np.load(npz_filename)
        return f['hrtf_locs'], f['hrtf_firs'], f['hrtf_sr']
    else:
        # Azimuths measured at +/- 40° elevation do not occur at integer angles
        azim_elev40 = np.linspace(0, 180, 56 // 2 + 1)
        list_fn_hrtf = []
        for elev in np.arange(-40, 91, 10, dtype=int):
            list_fn_hrtf.extend(sorted(glob.glob(f'kemar_hrtfs/elev{elev}/*wav')))
        hrtf_locs = []
        hrtf_firs = []
        for fn_hrtf in list_fn_hrtf:
            hrtf, hrtf_sr = sf.read(fn_hrtf)
            tmp = os.path.basename(fn_hrtf).replace('.wav', '')
            elev, azim = [float(_) for _ in tmp.strip('Ha').split('e')]
            if np.abs(elev) == 40:
                azim = azim_elev40[np.argmin(np.abs(azim_elev40 - azim))]
            hrtf_locs.append([1.4, azim, elev])
            hrtf_firs.append(hrtf)
        hrtf_locs = np.array(hrtf_locs)
        hrtf_firs = np.array(hrtf_firs)
        np.savez(
            npz_filename,
            hrtf_locs=hrtf_locs,
            hrtf_firs=hrtf_firs,
            hrtf_sr=hrtf_sr)
        print(f'[load_kemar_hrtfs] cache file for future calls: {npz_filename}')
    return hrtf_locs, hrtf_firs, hrtf_sr


def get_gm_hrtfs():
    """
    Old method of loading HRTFs from francl code
    """
    hrtf_locs = scipy.io.loadmat('HRTFs/data_locs.mat')['locs_gardnermartin']
    hrtf_locs = np.array(hrtf_locs, dtype=float)
    hrtf_firs = []
    hrtf_filenames = scipy.io.loadmat('HRTFs/file_names.mat')['gardnermartin_file']
    for fn in hrtf_filenames:
        hrtf, hrtf_sr = sf.read(fn.replace('\\', '/').replace(' ', ''))
        hrtf_firs.append(hrtf)
    hrtf_firs = np.array(hrtf_firs, dtype=float)
    return hrtf_locs, hrtf_firs, hrtf_sr

a, b, c = load_kemar_hrtfs()
aa, bb, cc = get_gm_hrtfs()

print(np.isclose(a, aa).all())
print(np.array_equal(b, bb))
print(c == cc)


True
True
True


In [ ]:
eng = matlab.engine.start_matlab();
if 'src/' not in eng.path():
    eng.addpath('src/');


In [ ]:
# fn = '/scratch2/weka/mcdermott/francl/Room_Simulator_20181115_Rebuild/Expanded_HRIRdist140-5deg_elev_az_room3x3y4z_materials23wall23floor26ciel/15elev_35az_1.40x1.60y2.00z_l.wav'
# room_materials = [23, 23, 23, 23, 23, 26]
# room_dim_xyz = [3, 3, 4]
# head_pos_xyz = [1.4, 1.6, 2.0]
# head_azim = 0
# src_azim = 35
# src_elev = 15
# src_dist = 1.4

fn = '/scratch2/weka/mcdermott/francl/Room_Simulator_20181115_Rebuild/Expanded_HRIRdist140-5deg_elev_az_room5x4y2z_materials1wall15floor16ciel/0elev_45az_3.60x1.40y2.00z_l.wav'
room_materials = [1, 1, 1, 1, 15, 16]
room_dim_xyz = [5, 4, 2]
head_pos_xyz = [3.6, 1.4, 2.0]
head_azim = 0
src_azim = 45
src_elev = 0
src_dist = 1.4

c = 344.5
buffer = 0
sr = 44100
dur = 0.5
use_hrtf_symmetry = True
use_highpass = True
use_jitter = False
incorporate_lead_zeros = False
use_log_distance = False
hrtf_locs = None
hrtf_firs = None

print(fn)
y_l, hrtf_sr = sf.read(fn)
y_r, hrtf_sr = sf.read(fn.replace('_l.wav', '_r.wav'))
brir_francl = np.stack([y_l, y_r], axis=1)
print(brir_francl.shape, hrtf_sr, sr)


In [ ]:
brir_matlab = simulator_matlab.get_brir(
    room_materials=room_materials,
    room_dim_xyz=room_dim_xyz,
    head_pos_xyz=head_pos_xyz,
    head_azim=head_azim,
    src_azim=src_azim,
    src_elev=src_elev,
    src_dist=src_dist,
    buffer=buffer,
    sr=sr,
    dur=dur,
    use_jitter=use_jitter,
    use_hrtf_symmetry=use_hrtf_symmetry,
    incorporate_lead_zeros=incorporate_lead_zeros,
    verbose=True,
    eng=eng)


In [ ]:
brir_msaddler = simulator.get_brir(
    room_materials=room_materials,
    room_dim_xyz=room_dim_xyz,
    head_pos_xyz=head_pos_xyz,
    head_azim=head_azim,
    src_azim=src_azim,
    src_elev=src_elev,
    src_dist=src_dist,
    buffer=buffer,
    sr=sr,
    dur=dur,
    use_jitter=use_jitter,
    use_hrtf_symmetry=use_hrtf_symmetry,
    incorporate_lead_zeros=incorporate_lead_zeros)


In [ ]:
fig, ax = plt.subplots(figsize=(16, 8), nrows=3, ncols=1)
ax[0].plot(brir_msaddler[:, 0], 'b')
ax[0].plot(brir_msaddler[:, 1], 'r')
ax[0].plot(brir_francl[:, 0], 'c', ls=':')
ax[0].plot(brir_francl[:, 1], 'm', ls=':')

ax[1].plot(brir_matlab[:, 0], 'b')
ax[1].plot(brir_matlab[:, 1], 'r')
ax[1].plot(brir_francl[:, 0], 'c', ls=':')
ax[1].plot(brir_francl[:, 1], 'm', ls=':')

ax[2].plot(brir_msaddler[:, 0] - brir_francl[:, 0], 'b')
ax[2].plot(brir_msaddler[:, 1] - brir_francl[:, 1], 'r')
ax[2].plot(brir_matlab[:, 0] - brir_francl[:, 0], 'c', ls=':')
ax[2].plot(brir_matlab[:, 1] - brir_francl[:, 1], 'm', ls=':')

for ax_ in ax:
    util_figures.format_axes(
        ax_,
        xlimits=[0, 500]
    )
plt.show()


In [ ]:
"""
DEBUGGING WORK
"""
room_materials = [1, 1, 1, 1, 15, 16]
room_dim_xyz = [7, 8, 4]
head_pos_xyz = [3.6, 1.4, 2.0]
head_azim = -49
src_azim = 145
src_elev = 20
src_dist = 1.4
buffer = 0.5

c = 344.5
buffer = 0
sr = 44100
dur = 0.5
use_hrtf_symmetry = True
use_highpass = True
use_jitter = False
incorporate_lead_zeros = False
use_log_distance = True
hrtf_locs = None
hrtf_firs = None

importlib.reload(simulator)

np.random.seed(0)
brir_msaddler = simulator.get_brir(
    room_materials=room_materials,
    room_dim_xyz=room_dim_xyz,
    head_pos_xyz=head_pos_xyz,
    head_azim=head_azim,
    src_azim=src_azim,
    src_elev=src_elev,
    src_dist=src_dist,
    buffer=buffer,
    sr=sr,
    c=c,
    dur=dur,
    hrtf_locs=hrtf_locs,
    hrtf_firs=hrtf_firs,
    use_hrtf_symmetry=use_hrtf_symmetry,
    use_log_distance=use_log_distance,
    use_jitter=use_jitter,
    use_highpass=use_highpass,
    incorporate_lead_zeros=incorporate_lead_zeros,
    processes=12,
    strict=True,
    verbose=True)


In [ ]:
np.random.seed(0)
brir_matlab = simulator_matlab.get_brir(
    room_materials=room_materials,
    room_dim_xyz=room_dim_xyz,
    head_pos_xyz=head_pos_xyz,
    head_azim=head_azim,
    src_azim=src_azim,
    src_elev=src_elev,
    src_dist=src_dist,
    buffer=buffer,
    sr=sr,
    c=c,
    dur=dur,
    use_hrtf_symmetry=use_hrtf_symmetry,
    use_log_distance=use_log_distance,
    use_jitter=use_jitter,
    use_highpass=use_highpass,
    incorporate_lead_zeros=incorporate_lead_zeros,
    eng=eng)


In [ ]:
fig, ax = plt.subplots(figsize=(16, 8), nrows=2, ncols=1)

offset_brir_msaddler = brir_msaddler[1:]
offset_brir_matlab = brir_matlab[:-1]

ax[0].plot(offset_brir_msaddler[:, 0], 'b')
ax[0].plot(offset_brir_msaddler[:, 1], 'r')
ax[0].plot(offset_brir_matlab[:, 0], 'c', ls=':')
ax[0].plot(offset_brir_matlab[:, 1], 'm', ls=':')

ax[1].plot(offset_brir_msaddler[:, 0] - offset_brir_matlab[:, 0], 'b')
ax[1].plot(offset_brir_msaddler[:, 1] - offset_brir_matlab[:, 1], 'r')

for ax_ in ax:
    util_figures.format_axes(
        ax_,
#         xlimits=[0, 8000],
#         ylimits=[-0.01, 0.01]
    )
plt.show()


In [ ]:
# import cProfile
# import pstats
# importlib.reload(port)

# func_to_profile = lambda : port.get_brir(
#     room_materials=[23, 23, 23, 23, 23, 26],
#     room_dim_xyz=[3, 3, 4],
#     head_pos_xyz=[1.4, 1.6, 2.0],
#     head_azim=0,
#     src_azim=35,
#     src_elev=15,
#     src_dist=1.4,
#     sr=44100,
#     dur=0.5,
#     use_jitter=False,
#     use_hrtf_symmetry=True,
#     incorporate_lead_zeros=False)

# cProfile.run('func_to_profile()', 'profiler_stats')

# p = pstats.Stats('profiler_stats')
# p.strip_dirs().sort_stats('cumulative').print_stats()


In [ ]:
hrtf_locs = scipy.io.loadmat('HRTFs/data_locs.mat')['locs_gardnermartin']
hrtf_locs = np.array(hrtf_locs, dtype=float)
hrtf_locs

fig, ax = plt.subplots()
x = hrtf_locs[:, 1]
y = hrtf_locs[:, 2]
ax.plot(x, y, 'k.')
plt.show()
